In [1]:
import pandas as pd
import pyspark
import os 
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf      # sf = spark functions
import pyspark.sql.types as st          # st = spark types
import datetime as dt

# spark = SparkSession.builder.master('local').getOrCreate()

# spark = SparkSession.builder.getOrCreate()

In [73]:
data_dir = './data'

cpi_file = 'US-CPI.csv'
unemp_file = 'USUnemployment.csv'

c_schema = 'Yearmon string, CPI float'
u_schema = 'Year int, Jan float ,Feb float ,Mar float,Apr float ,May float,Jun float ,Jul float ,Aug float ,Sep float,Oct float,Nov float,Dec float'

cpi_df = spark.read.csv(os.path.join(data_dir, cpi_file), schema=c_schema, header=True, enforceSchema=True)
unemp_df = spark.read.csv(os.path.join(data_dir, unemp_file), schema=u_schema, header=True, enforceSchema=True)

cpi_df.show(5)
unemp_df.show(5)

+----------+---+
|   Yearmon|CPI|
+----------+---+
|01-01-1913|9.8|
|01-02-1913|9.8|
|01-03-1913|9.8|
|01-04-1913|9.8|
|01-05-1913|9.7|
+----------+---+
only showing top 5 rows

+----+---+---+---+---+---+---+---+---+---+---+---+---+
|Year|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|
+----+---+---+---+---+---+---+---+---+---+---+---+---+
|1948|3.4|3.8|4.0|3.9|3.5|3.6|3.6|3.9|3.8|3.7|3.8|4.0|
|1949|4.3|4.7|5.0|5.3|6.1|6.2|6.7|6.8|6.6|7.9|6.4|6.6|
|1950|6.5|6.4|6.3|5.8|5.5|5.4|5.0|4.5|4.4|4.2|4.2|4.3|
|1951|3.7|3.4|3.4|3.1|3.0|3.2|3.1|3.1|3.3|3.5|3.5|3.1|
|1952|3.2|3.1|2.9|2.9|3.0|3.0|3.2|3.4|3.1|3.0|2.8|2.7|
+----+---+---+---+---+---+---+---+---+---+---+---+---+
only showing top 5 rows



In [74]:
from pyspark.sql.functions import col,avg
# rename cols in cpi df

for column in cpi_df.columns: 
  cpi_df = cpi_df.withColumnRenamed(column, column.lower())

cpi_df = cpi_df.withColumnRenamed('yearmon', 'year')

# split year column into month, date, year columns
split_cpi_df = cpi_df.withColumn("month", sf.split(sf.col('year'), "-").getItem(0))\
              .withColumn('date', sf.split(sf.col('year'), "-").getItem(1))\
              .withColumn('year', sf.split(sf.col('year'), "-").getItem(2))

# cast year col to int for groupby method
# cpi_df = cpi_df.withColumn('year', sf.col('year').cast('int'))

# rename cols in unemployment df
for column in unemp_df.columns: 
  unemp_df = unemp_df.withColumnRenamed(column, column.lower())

# calculate avg unemployment rate per year
udf_avg = sf.udf(lambda array: sum(array)/len(array))
unemp_df = unemp_df.withColumn("avg_unemp_rate_per_year", udf_avg(sf.array('Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec')).cast('float'))

# calculate avg inflation rate per year
avg_cpi_df = split_cpi_df.groupBy(sf.col('year')) \
                .agg(sf.avg(sf.col('cpi')).alias('avg_cpi_per_year'))

# avg_cpi_df.show(truncate=False)
avg_cpi_df.show(5)
unemp_df.show(5)

+----+------------------+
|year|  avg_cpi_per_year|
+----+------------------+
|1953|26.766666571299236|
|1957|28.091666380564373|
|1987|           113.625|
|1956|27.183333079020183|
|1936|13.866666634877523|
+----+------------------+
only showing top 5 rows

+----+---+---+---+---+---+---+---+---+---+---+---+---+-----------------------+
|year|jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec|avg_unemp_rate_per_year|
+----+---+---+---+---+---+---+---+---+---+---+---+---+-----------------------+
|1948|3.4|3.8|4.0|3.9|3.5|3.6|3.6|3.9|3.8|3.7|3.8|4.0|                   3.75|
|1949|4.3|4.7|5.0|5.3|6.1|6.2|6.7|6.8|6.6|7.9|6.4|6.6|                   6.05|
|1950|6.5|6.4|6.3|5.8|5.5|5.4|5.0|4.5|4.4|4.2|4.2|4.3|              5.2083335|
|1951|3.7|3.4|3.4|3.1|3.0|3.2|3.1|3.1|3.3|3.5|3.5|3.1|              3.2833333|
|1952|3.2|3.1|2.9|2.9|3.0|3.0|3.2|3.4|3.1|3.0|2.8|2.7|                  3.025|
+----+---+---+---+---+---+---+---+---+---+---+---+---+-----------------------+
only showing top 5 rows



In [2]:
data_dir = 'dsa-airflow/data'

cpi_file = 'US-CPI.csv'
unemp_file = 'USUnemployment.csv'

# read and rename cpi file / parse_dates=['Yearmon']
cpi  = pd.read_csv(os.path.join(data_dir, cpi_file), header=0)
cpi = cpi.rename(columns={'Yearmon': 'year', 'CPI': 'cpi'})
cpi[['month', 'date', 'year']] = cpi.year.str.split("-", expand=True)

# read and rename unemployment file
unemp = pd.read_csv(os.path.join(data_dir, unemp_file), header=0)
unemp.columns = unemp.columns.str.lower()


# calculate the avg yearly unemp. rate
columns = ['jan',	'feb',	'mar',	'apr',	'may'	,'jun',	'jul',	'aug',	'sep',	'oct',	'nov',	'dec']
unemp['avg_unemp_per_year']  = unemp[columns].mean(axis=1)
avg_unemp_per_year  = unemp[['year', 'avg_unemp_per_year']]

# calculate the avg yearly cpi rate
avg_cpi_per_year = cpi.groupby('year').mean('cpi')
avg_cpi_per_year = avg_cpi_per_year.reset_index()
avg_cpi_per_year['year'] = avg_cpi_per_year['year'].astype('int')

print(avg_unemp_per_year.head(5))
print(avg_unemp_per_year.dtypes)
print(avg_cpi_per_year.head(10))
print(avg_cpi_per_year.dtypes)
# unemp.dtypes


   year  avg_unemp_per_year
0  1948            3.750000
1  1949            6.050000
2  1950            5.208333
3  1951            3.283333
4  1952            3.025000
year                    int64
avg_unemp_per_year    float64
dtype: object
   year        cpi
0  1913   9.883333
1  1914  10.016667
2  1915  10.108333
3  1916  10.883333
4  1917  12.825000
5  1918  15.041667
6  1919  17.333333
7  1920  20.041667
8  1921  17.850000
9  1922  16.750000
year      int64
cpi     float64
dtype: object


In [3]:
from google.cloud import bigquery
from google.oauth2 import service_account

# create full table id
PROJECT_ID = "team-week-3"
DATASET_ID = "inflation_unemployment"

# create bigquery client
key_path = os.path.expanduser("/home/chloe_ycl/.creds/team-week-3.json")
credentials = service_account.Credentials.from_service_account_file(key_path, 
                                                              scopes=["https://www.googleapis.com/auth/cloud-platform"])

client = bigquery.Client(credentials=credentials, project=credentials.project_id)

print("Successfully created a BiqQuery client")
print(f"Project: {client.project}")

# create dataset
dataset_id = "{}.inflation_unemployment".format(client.project)

#construct dataset obj to send to API
dataset = bigquery.Dataset(dataset_id)

#specify the location
dataset.location = "US"

dataset = client.create_dataset(dataset, timeout=30, exists_ok=True) #Make an API request
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Successfully created a BiqQuery client
Project: team-week-3
Created dataset team-week-3.inflation_unemployment


In [4]:
# funtion to load table from dataframes

def load_table(
    df: pd.DataFrame, 
    client: bigquery.Client, 
    table_name: str, 
    schema: bigquery.SchemaField,
    create_disposition: str = 'CREATE_IF_NEEDED', 
    write_disposition: str = 'WRITE_TRUNCATE'
    ) -> None:
    
    job_config = bigquery.LoadJobConfig(
        create_disposition=create_disposition,
        write_disposition=write_disposition,
        schema=schema
    )
    
    job = client.load_table_from_dataframe(df, destination=table_name, job_config=job_config)
    job.result()        # wait for the job to finish



# create our own bigquery schema
CPI_METADATA = {
    'cpi_rates': {
        'table_name': 'cpi_rates',
        'schema' : [
            bigquery.SchemaField("year", "INTEGER", mode="REQUIRED"),
            bigquery.SchemaField("cpi", "FLOAT", mode="REQUIRED")]
    }}

UNEMP_METADATA = {
    'unemployment_rates' :{
        'table_name': 'unemployment_rates',
        'schema': [
            bigquery.SchemaField("year", "INTEGER", mode="REQUIRED"),
            bigquery.SchemaField("avg_unemp_per_year", "FLOAT", mode="REQUIRED")]
    }}

In [5]:
# load to bq
table_name = f"{PROJECT_ID}.{DATASET_ID}.{CPI_METADATA['cpi_rates']['table_name']}"
schema = CPI_METADATA['cpi_rates']['schema']
load_table(avg_cpi_per_year, client, table_name, schema)

table_name = f"{PROJECT_ID}.{DATASET_ID}.{UNEMP_METADATA['unemployment_rates']['table_name']}"
schema = UNEMP_METADATA['unemployment_rates']['schema']
load_table(avg_unemp_per_year, client, table_name, schema)

In [10]:
# query Stock file from bq
STOCK_DATASET_ID = 'tech_stocks_world_events'

stocks = f""" SELECT * FROM {PROJECT_ID}.{STOCK_DATASET_ID}.stocks """

result = client.query(stocks)

# read to dataframe
df = result.to_dataframe()

# read to csv file
df.to_csv(os.path.join(data_dir, 'stocks.csv'), header=True)

# print(stocks_df.dtypes)
# stocks_df
